# M5 Forecasting Challenge: Prophet Model
#### Author: Stacy Liu

## Introduction

**Prophet Model**: Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well. (https://facebook.github.io/prophet/)


**The goal of Prophet Notebook**: This notebook will be used to predict sales using Prophet

**The data**: We are working with **42,840 hierarchical time series**. The data were obtained in the 3 US states of California (CA), Texas (TX), and Wisconsin (WI). “Hierarchical” here means that data can be aggregated on different levels: item level, department level, product category level, and state level. The sales information reaches back from Jan 2011 to June 2016. In addition to the sales numbers, we are also given corresponding data on prices, promotions, and holidays. Note, that we have been warned that **most of the time series contain zero values.**

The data comprises **3049** individual products from *3 categories* and *7 departments*, sold in 10 stores in 3 states. The hierachical aggregation captures the combinations of these factors. For instance, we can create 1 time series for all sales, 3 time series for all sales per state, and so on. The largest category is sales of all individual 3049 products per 10 stores for 30490 time series.

The training data comes in the shape of 4 separate files:

- sales_train.csv: this is our main training data.  Contains the historical daily unit sales data per product and store from d_1 - d_1913.

- sell_prices.csv: the store and item IDs together with the sales price of the item as a weekly average.

- calendar.csv: dates together with related features like day-of-the week, month, year, and an 3 binary flags for whether the stores in each state allowed purchases with SNAP food stamps at this date (1) or not (0).

- sales_train_evaluation.csv - Available one month before the competition deadline. It will include sales from d_1 - d_1941.

**The metrics:**

The point forecast submission are being evaluated using the **Root Mean Squared Scaled Error (RMSSE)**, which is derived from the Mean Absolute Scaled Error (MASE) that was designed to be scale invariant and symmetric. 

**Model & Results:**

Since at the item level data is fairly sporatic, I used a top down approach to predict 70 time series group by Department, Category, Store and State and then distribute down to the item level by weight (total sale in last 28 days). 

The following model metrics produced the smallest error of **0.57504** on the test set:
- Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.7, changepoint_range=0.8, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20)
    - Note that "holidays" includes events, snap dates, as well as a +2 and -3 window for events (from EDA we found that sales increased 2 days prior to event and decreases 3 days after)

## Initial Data Loading

In [1]:
# Import libraries
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pickle
import os
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation

%matplotlib inline

Importing plotly failed. Interactive plots will not work.


## **Skip to 'Apply Prophet' section for model testing**

To speed up modeling, I'll be fitting the model and predicting sales for the top 100 revenue (sales price * sales) generating items from each category from the last 28 days (from 3/28/2016 to 4/24/2016).

Run the below code to reduce memory usage when importing cvs

In [ ]:
def reduce_mem_usage(df, verbose=True):
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if   c_min > np.iinfo(np.int8 ).min and c_max < np.iinfo(np.int8).max :
                    df[col] = df[col].astype(np.int8 )
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if   c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
        
    return df

In [ ]:
# Load saved sales data
filename = os.path.join("sales.csv")
sales = reduce_mem_usage(pd.read_csv(filename))
sales.head()

In [ ]:
sales.shape

To speed up modeling, I'll be fitting the model and predicting sales for the top 100 revenue (sales price * sales) generating items from each category from the last 28 days (from 3/28/2016 to 4/24/2016).

In [ ]:
sales['date']=pd.to_datetime(sales['date'])

# pull sales from the last 28 days of the training data
sales_last28days = sales[sales['date']>=pd.to_datetime('20160328')]
sales_last28days = sales_last28days[sales_last28days['date']<=pd.to_datetime('20160424')]

# calculate revenue
sales_last28days['revenue'] = sales_last28days['sales'] * sales_last28days['sell_price']

# get dataframe sorted by revenue for each category 
top_100 = sales_last28days.groupby(['cat_id', 'id'], as_index=False).sum()
top_100 = top_100.groupby(['cat_id']).apply(lambda x: x.sort_values(["revenue"], ascending = False)).reset_index(drop=True)
# select top 100 rows within each category
top_100 = top_100.groupby('cat_id').head(100)
# pull the IDs
top_100 = top_100['id']

In [ ]:
# save to file
top_100.to_csv("top_100_items.csv", sep=",", index=False)

In [ ]:
filename = os.path.join("top_100_items.csv")
top_100 = pd.read_csv(filename)

# pull sales data of the top 100 items
top_100_sales = sales.merge(top_100, on = ['id'], how = 'inner')

In [ ]:
# sort and save to file
top_100_sales = top_100_sales.groupby(['id']).apply(lambda x: x.sort_values(["date"], ascending = True)).reset_index(drop=True)
top_100_sales.to_csv("top_100_sales.csv", sep=",", index=False)

## Apply simple Prophet on 300 items

In [ ]:
# Load saved top 100 sales data
filename = os.path.join("top_100_sales.csv")
top_100_sales = pd.read_csv(filename)
top_100_sales['date']=pd.to_datetime(top_100_sales['date'])
top_100_sales.tail()

In [ ]:
# split into training and testing
cut_off_date = pd.to_datetime('20160424')

train = top_100_sales[top_100_sales['date']<=cut_off_date]
test = top_100_sales[top_100_sales['date'] > cut_off_date]

print(train.shape)
print(test.shape)

In [ ]:
# see how many rows we have of each item
train.id.value_counts()

In [ ]:
# create dataframe to save results
results1 = pd.DataFrame(columns=test['date'].unique())
results1.insert(0, 'id', test['id'].unique())
results1

In [ ]:
train = train.pivot(index='id', columns='date', values = 'sales').reset_index()

print(dt.datetime.now())
for i in range(len(train)):
    df1 = train.loc[[i]].copy()
    df1 = df1.drop(columns=['id'])
    df1 = df1.T.reset_index().drop(columns="date")
    df1 = df1.rename(columns={i:"y"})
    df1["ds"] = pd.Series(pd.date_range('2011-01-29', periods=len(df1)))
    df1 = df1[["ds", "y"]]
    model = Prophet(uncertainty_samples=False, daily_seasonality=True)
    model.fit(df1)
    future_data=model.make_future_dataframe(periods=28).tail(28)
    forecast_data=model.predict(future_data)
    results1.iloc[i,1:] = forecast_data['yhat'].iloc[-28:].values
    if i == 300: 
       # print(sales_train.loc[[i], "id"])
       # print(forecast_data[['ds', 'yhat']].head(28))
        break

#transform results for evaluation
results1.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
results1_transposed = pd.melt(results1,
             id_vars = ['id'],
             value_vars = [col for col in results.columns if col.startswith("F")],
             var_name = "d",
             value_name = "sales")
print(dt.datetime.now())

In [ ]:
# Evaluate results
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
MSE = mean_squared_error(y_true = test.sales.values, y_pred = results1_transposed.sales.values)
RMSE = np.sqrt(mean_squared_error(test.sales.values, results1_transposed.sales.values))
print(MSE)
print(RMSE)

# Mean Forecast Error
forecast_errors = [test.sales.values[i]-results1_transposed.sales.values[i] for i in range(len(test))]
bias = sum(forecast_errors) * 1.0/len(test)
print('Bias: %f' % bias)

#MAPE
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

mape = mean_absolute_percentage_error(test.sales.values, results1_transposed.sales.values)
print('MAPE: %f' % mape)

Baseline result:
    - Model runtime: 7min
    - MSE: 531.0445125981449
    - RMSE: 23.044403064478477
    - Bias: 0.675911
    - MAPE: 258.782741
Bias result is positive, meanng that we have under forecast.

## Tune Model Inputs

**Remove Christmas**

In [ ]:
train_modified = train.copy()
train_modified = train_modified.drop(columns=[pd.to_datetime('20111225'),
                                             pd.to_datetime('20121225'),
                                             pd.to_datetime('20131225'),
                                             pd.to_datetime('20141225'),
                                             pd.to_datetime('20151225'),], axis=1)

**Add Holiday & SNAP**

In [2]:
# Import calendar
filename = os.path.join("Data", "m5-forecasting-accuracy-updated", "calendar.csv")
calendar = pd.read_csv(filename)

In [3]:
# Holiday
calendar['ds'] = pd.to_datetime(calendar['date'])
events1 = pd.Series(calendar['event_name_1'].values, index=calendar['ds'].values).dropna()
events2 = pd.Series(calendar['event_name_2'].values, index=calendar['ds'].values).dropna()
events = pd.DataFrame(pd.concat([events1, events2], axis=0))
events['ds'] = events.index.values
events.rename({0: 'holiday'}, axis=1, inplace=True)
events.reset_index(drop=True, inplace=True)

#SNAP
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': calendar[calendar['snap_CA'] == 1]['date']})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': calendar[calendar['snap_TX'] == 1]['date']})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': calendar[calendar['snap_WI'] == 1]['date']})
holidays = pd.concat((events, df_ev_3, df_ev_4, df_ev_5))

holidays['ds'] = pd.to_datetime(holidays['ds'])

holidays

,holiday,ds
0,SuperBowl,2011-02-06
1,ValentinesDay,2011-02-14
2,PresidentsDay,2011-02-21
3,LentStart,2011-03-09
4,LentWeek2,2011-03-16
...,...,...
1958,snap_WI,2016-06-09
1960,snap_WI,2016-06-11
1961,snap_WI,2016-06-12
1963,snap_WI,2016-06-14


## Run Model with tuned inputs

In [ ]:
print(dt.datetime.now())
for i in range(len(train_modified)):
    df1 = train.loc[[i]].copy()
    df1 = df1.drop(columns=['id'])
    df1 = df1.T.reset_index().drop(columns="date")
    df1 = df1.rename(columns={i:"y"})
    df1["ds"] = pd.Series(pd.date_range('2011-01-29', periods=len(df1)))
    df1 = df1[["ds", "y"]]
    model = Prophet(holidays = holidays, uncertainty_samples=False, 
                    daily_seasonality=True, n_changepoints = 50, 
                    changepoint_range = 0.8, changepoint_prior_scale = 0.7)
    model.fit(df1)
    future_data=model.make_future_dataframe(periods=28).tail(28)
    forecast_data=model.predict(future_data)
    results.iloc[i,1:] = forecast_data['yhat'].iloc[-28:].values
    if i == 300: 
       # print(sales_train.loc[[i], "id"])
       # print(forecast_data[['ds', 'yhat']].head(28))
        break

#transform results for evaluation
results.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
results_transposed = pd.melt(results,
             id_vars = ['id'],
             value_vars = [col for col in results.columns if col.startswith("F")],
             var_name = "d",
             value_name = "sales")
print(dt.datetime.now())

In [ ]:
# evaluate result
MSE = mean_squared_error(y_true = test.sales.values, y_pred = results_transposed.sales.values)
RMSE = np.sqrt(mean_squared_error(test.sales.values, results_transposed.sales.values))
print(MSE)
print(RMSE)

# Mean Forecast Error
forecast_errors = [test.sales.values[i]-results_transposed.sales.values[i] for i in range(len(test))]
bias = sum(forecast_errors) * 1.0/len(test)
print('Bias: %f' % bias)

#MAPE
mape = mean_absolute_percentage_error(test.sales.values, results_transposed.sales.values)
print('MAPE: %f' % mape)

Result with Holiday & Snap and some model tuning: Similiar MSE/RMSE as the baseline model but lower bias
    - Model run time: 47 min
    - MSE: 542.2541423909882
    - RMSE: 23.28635098917364
    - Bias: 0.454468

**Add window to calendar since we know from EDA that sales increase 1-2 days prior to events and decreases 3 days after**

In [4]:
# Holiday
calendar['ds'] = pd.to_datetime(calendar['date'])
events1 = pd.Series(calendar['event_name_1'].values, index=calendar['ds'].values).dropna()
events2 = pd.Series(calendar['event_name_2'].values, index=calendar['ds'].values).dropna()
events = pd.DataFrame(pd.concat([events1, events2], axis=0))
events['ds'] = events.index.values
events.rename({0: 'holiday'}, axis=1, inplace=True)
events.reset_index(drop=True, inplace=True)

#add lower and upper window
events = pd.DataFrame({
  'holiday': events['holiday'],
  'ds': events['ds'],
  'lower_window': -2,
  'upper_window': 3
})

#SNAP
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': calendar[calendar['snap_CA'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': calendar[calendar['snap_TX'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': calendar[calendar['snap_WI'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
holidays = pd.concat((events, df_ev_3, df_ev_4, df_ev_5))

holidays['ds'] = pd.to_datetime(holidays['ds'])

In [5]:
holidays

,holiday,ds,lower_window,upper_window
0,SuperBowl,2011-02-06,-2,3
1,ValentinesDay,2011-02-14,-2,3
2,PresidentsDay,2011-02-21,-2,3
3,LentStart,2011-03-09,-2,3
4,LentWeek2,2011-03-16,-2,3
...,...,...,...,...
1958,snap_WI,2016-06-09,0,0
1960,snap_WI,2016-06-11,0,0
1961,snap_WI,2016-06-12,0,0
1963,snap_WI,2016-06-14,0,0


In [ ]:
# simplified holiday
events_1 = pd.DataFrame({'holiday': 'Event 1', 'ds': calendar[calendar['snap_CA'] == 1]['date']})
events_2 = pd.DataFrame({'holiday': 'Event 2', 'ds': calendar[calendar['snap_CA'] == 1]['date']})
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': calendar[calendar['snap_CA'] == 1]['date']})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': calendar[calendar['snap_TX'] == 1]['date']})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': calendar[calendar['snap_WI'] == 1]['date']})
holidays_simplified = pd.concat((events_1, events_2, df_ev_3, df_ev_4, df_ev_5))


In [ ]:
# simplified holiday with window
events_1 = pd.DataFrame({'holiday': 'Event 1', 'ds': calendar[calendar['snap_CA'] == 1]['date'], 'lower_window': -2, 'upper_window': 3})
events_2 = pd.DataFrame({'holiday': 'Event 2', 'ds': calendar[calendar['snap_CA'] == 1]['date'], 'lower_window': -2, 'upper_window': 3})
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': calendar[calendar['snap_CA'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': calendar[calendar['snap_TX'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': calendar[calendar['snap_WI'] == 1]['date'], 'lower_window': 0, 'upper_window': 0})
holidays_simplified_window = pd.concat((events_1, events_2, df_ev_3, df_ev_4, df_ev_5))


## Run model with upper and lower window

In [ ]:
print(dt.datetime.now())
for i in range(len(train_modified)):
    df1 = train.loc[[i]].copy()
    df1 = df1.drop(columns=['id'])
    df1 = df1.T.reset_index().drop(columns="date")
    df1 = df1.rename(columns={i:"y"})
    df1["ds"] = pd.Series(pd.date_range('2011-01-29', periods=len(df1)))
    df1 = df1[["ds", "y"]]
    model = Prophet(holidays = holidays, uncertainty_samples=False, 
                    daily_seasonality=True, n_changepoints = 50, 
                    changepoint_range = 0.8, changepoint_prior_scale = 0.7)
    model.fit(df1)
    future_data=model.make_future_dataframe(periods=28).tail(28)
    forecast_data=model.predict(future_data)
    results.iloc[i,1:] = forecast_data['yhat'].iloc[-28:].values
    print(i)
    if i == 300: 
       # print(sales_train.loc[[i], "id"])
       # print(forecast_data[['ds', 'yhat']].head(28))
        break

#transform results for evaluation
results.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
results_transposed = pd.melt(results,
             id_vars = ['id'],
             value_vars = [col for col in results.columns if col.startswith("F")],
             var_name = "d",
             value_name = "sales")
print(dt.datetime.now())

In [ ]:
# evaluate result
MSE = mean_squared_error(y_true = test.sales.values, y_pred = results_transposed.sales.values)
RMSE = np.sqrt(mean_squared_error(test.sales.values, results_transposed.sales.values))
print(MSE)
print(RMSE)

# Mean Forecast Error
forecast_errors = [test.sales.values[i]-results_transposed.sales.values[i] for i in range(len(test))]
bias = sum(forecast_errors) * 1.0/len(test)
print('Bias: %f' % bias)

#MAPE
mape = mean_absolute_percentage_error(test.sales.values, results_transposed.sales.values)
print('MAPE: %f' % mape)

Result with Holiday (with upper and lower window) & Snap and some model tuning: Higher MSE/RMSE as the baseline model and negative bias, meaning I've overforcasted 

- Model run time: 3hr15min
- MSE: 601.0255994065685
- RMSE: 24.515823449490096
- Bias: -0.334648

**Combine result1 and result so that Holiday is only applied to food and household**

In [ ]:
results_food = results_transposed.iloc[0:2800]
results_household = results_transposed.iloc[5600:8401]
results1_hobbies = results1_transposed.iloc[2800:5600]
results_combined = pd.concat((results_food, results_household, results1_hobbies))

In [ ]:
# evaluate result
MSE = mean_squared_error(y_true = test.sales.values, y_pred = results_combined.sales.values)
RMSE = np.sqrt(mean_squared_error(test.sales.values, results_combined.sales.values))
print(MSE)
print(RMSE)

# Mean Forecast Error
forecast_errors = [test.sales.values[i]-results_combined.sales.values[i] for i in range(len(test))]
bias = sum(forecast_errors) * 1.0/len(test)
print('Bias: %f' % bias)

#mape
mape = mean_absolute_percentage_error(test.sales.values, results_combined.sales.values)
print('MAPE: %f' % mape)

Result with Holiday (with upper and lower window) & Snap and some model tuning on ONLY food and household and baseline model for hobbies: similiar MSE/RMSE as the baseline model but much lower bias

    - Model run time: N/A
    - MSE: 564.4136245161772
    - RMSE: 23.75739094505491
    - Bias: 0.199190
    - MAPE: 286.449053
    
--> **Tuned model takes too long to run. Try a top down approach to limit number of runs**

## **Top down approach**
 - Predict 70 time series group by Department, Category, Store and State and then distribute down to the item level by weight (total sale in last 28 days)

### Evaluation

In [6]:
filename = os.path.join("Data",'m5-forecasting-accuracy-updated',"sales_train_validation.csv")
sales = pd.read_csv(filename)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [7]:
date_columns = sales.columns[sales.columns.str.contains("d_")]
date_columns

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1913)

In [8]:
#get all the dates
dates_s = [pd.to_datetime(calendar.loc[calendar['d'] == str_date,'date'].values[0]) for str_date in date_columns]

In [9]:
# sum sales by dept and store
df_sale_group_item = sales[np.hstack([['dept_id','store_id'],date_columns])].groupby(['dept_id','store_id']).sum()
df_sale_group_item = df_sale_group_item.reset_index()

In [10]:
def CreateTimeSeries(dept_id, store_id):
    item_series =  df_sale_group_item[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
    dates = pd.DataFrame({'ds': dates_s}, index=range(len(dates_s)))
    dates['y'] = item_series[date_columns].values[0].transpose()     
    return dates

In [12]:
def run_prophet(dept_id, store_id):
    timeserie = CreateTimeSeries(dept_id, store_id) 
    model = Prophet(growth='linear', 
                    holidays = holidays, 
                    uncertainty_samples=False, 
                    n_changepoints = 50, 
                    changepoint_prior_scale=0.7, 
                    changepoint_range=0.8, 
                    holidays_prior_scale=20, 
                    seasonality_mode='multiplicative', 
                    seasonality_prior_scale=20,
                    daily_seasonality=False)
   # model.add_seasonality(name='monthly', period=365.25/12, fourier_order=55)
    # model.add_seasonality(name='quarterly', period=365.25/4, fourier_order=5)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(forecast)
    return np.append(np.array([dept_id,store_id]),forecast['yhat'].values.transpose())

In [13]:
# create list param
ids = []
for i in range(0,df_sale_group_item.shape[0]):
    ids = ids + [(df_sale_group_item[i:i+1]['dept_id'].values[0],df_sale_group_item[i:i+1]['store_id'].values[0])]

In [14]:
predictions = []
for i in range(len(ids)):
    results = list(run_prophet(ids[i][0],ids[i][1]))
    predictions.append(results)
    print(i)
    if i == len(ids)+1: 
        break

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [15]:
#Submission
df_sub_eva = pd.DataFrame()
for k in range(0, len(predictions)):
    dept_id = predictions[k][0]
    store_id = predictions[k][1]

    df_item = sales.loc[(sales.dept_id == dept_id) & (sales.store_id == store_id)][['id']]
    df_item['val'] = sales[(sales.dept_id == dept_id) & (sales.store_id == store_id)].iloc[:, np.r_[0,-28:0]].sum(axis = 1)
    for i in range(1,29):
        df_item[f'F{i}'] = (df_item['val'] * float(predictions[k][i+1]) / df_item['val'].sum())
    df_sub_eva = pd.concat([df_sub_eva, df_item])

df_sub_eva = df_sub_eva.drop('val',axis=1)

### Validation

In [16]:
filename = os.path.join("Data",'m5-forecasting-accuracy-updated',"sales_train_evaluation.csv")
sales = pd.read_csv(filename)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [17]:
date_columns = sales.columns[sales.columns.str.contains("d_")]
date_columns

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938',
       'd_1939', 'd_1940', 'd_1941'],
      dtype='object', length=1941)

In [18]:
#get all the dates
dates_s = [pd.to_datetime(calendar.loc[calendar['d'] == str_date,'date'].values[0]) for str_date in date_columns]

In [19]:
# sum sales by dept and store
df_sale_group_item = sales[np.hstack([['dept_id','store_id'],date_columns])].groupby(['dept_id','store_id']).sum()
df_sale_group_item = df_sale_group_item.reset_index()

In [20]:
# create list param
ids = []
for i in range(0,df_sale_group_item.shape[0]):
    ids = ids + [(df_sale_group_item[i:i+1]['dept_id'].values[0],df_sale_group_item[i:i+1]['store_id'].values[0])]

In [21]:
predictions_val = []
for i in range(len(ids)):
    results = list(run_prophet(ids[i][0],ids[i][1]))
    predictions_val.append(results)
    print(i)
    if i == len(ids)+1: 
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [22]:
#Submission
df_sub_val = pd.DataFrame()
for k in range(0, len(predictions_val)):
    dept_id = predictions_val[k][0]
    store_id = predictions_val[k][1]

    df_item = sales.loc[(sales.dept_id == dept_id) & (sales.store_id == store_id)][['id']]
    df_item['val'] = sales[(sales.dept_id == dept_id) & (sales.store_id == store_id)].iloc[:, np.r_[0,-28:0]].sum(axis = 1)
    for i in range(1,29):
        df_item[f'F{i}'] = (df_item['val'] * float(predictions_val[k][i+1]) / df_item['val'].sum())
    df_sub_val = pd.concat([df_sub_val, df_item])

df_sub_val = df_sub_val.drop('val',axis=1)

In [24]:
#concat evaluation and validation rows
submission = pd.concat((df_sub_eva, df_sub_val), sort=False)
submission = submission.sort_values('id').reset_index(drop = True)

In [25]:
submission.to_csv('prophet_final.csv', index = False)
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.748841,0.745762,0.749093,0.787551,0.901222,1.087791,1.021491,0.823680,0.578717,...,0.918364,1.007640,0.910747,0.735664,0.720100,0.717941,0.751455,0.994230,1.013263,0.830553
1,FOODS_1_001_CA_1_validation,1.021954,1.006281,1.000192,0.883486,1.069442,1.399348,1.304984,1.040742,1.142979,...,1.331049,1.558718,1.416950,1.137125,1.140048,1.151739,1.227277,1.425870,1.657224,1.512108
2,FOODS_1_001_CA_2_evaluation,0.554705,0.573625,0.599493,0.636138,0.721050,0.876143,0.720564,0.834046,0.717240,...,0.769586,0.936655,0.859664,0.621730,0.637832,0.661010,0.686456,0.773635,0.920755,0.801782
3,FOODS_1_001_CA_2_validation,0.851491,0.878169,0.913496,0.984459,1.137863,1.513834,1.306525,0.863447,0.785336,...,1.186151,1.524956,1.389607,0.941473,0.983431,1.034354,1.109787,1.273592,1.607171,1.471676
4,FOODS_1_001_CA_3_evaluation,0.877740,0.887814,0.886819,0.914589,0.953661,1.077233,1.128418,1.075661,0.794235,...,1.045239,1.148215,1.131104,0.874695,0.865258,0.872541,0.860069,1.051363,1.141122,1.026848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_WI_1_validation,0.028825,0.028327,0.028833,0.025941,0.041314,0.046312,0.042362,0.028047,0.027732,...,0.035626,0.043900,0.039497,0.028692,0.028499,0.029318,0.030573,0.036836,0.045228,0.040877
60976,HOUSEHOLD_2_516_WI_2_evaluation,0.061786,0.060831,0.062314,0.066730,0.075641,0.071002,0.059261,0.061239,0.078423,...,0.080847,0.085555,0.077610,0.064538,0.060454,0.063132,0.067921,0.081902,0.083973,0.069620
60977,HOUSEHOLD_2_516_WI_2_validation,0.030469,0.030061,0.030985,0.033139,0.042432,0.047958,0.043909,0.032308,0.029081,...,0.038548,0.041075,0.037565,0.030258,0.029530,0.030157,0.032267,0.036778,0.041014,0.036889
60978,HOUSEHOLD_2_516_WI_3_evaluation,0.063748,0.061603,0.060655,0.064715,0.079290,0.081049,0.068920,0.067294,0.070833,...,0.082487,0.090608,0.086524,0.065533,0.061737,0.061458,0.064321,0.080629,0.080893,0.074558


### Top down model tuning:
1. Prophet(holidays = holidays_simplified, uncertainty_samples = False, n_changepoints = 50, changepoint_range = 0.8, changepoint_prior_scale = 0.7, seasonality_mode = 'multiplicative'): **0.64537**
2. Prophet(holidays = holidays_simplified_window, uncertainty_samples = False, n_changepoints = 50, changepoint_range = 0.8, changepoint_prior_scale = 0.7, seasonality_mode = 'multiplicative'): **0.63286**
3. Prophet(holidays = holidays, uncertainty_samples = False, n_changepoints = 50, changepoint_range = 0.8, changepoint_prior_scale = 0.7, seasonality_mode = 'multiplicative'): **0.57835**
4. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=30, changepoint_range=0.8, holidays_prior_scale=20, yearly_seasonality=2, daily_seasonality=False, weekly_seasonality=1, seasonality_mode='multiplicative', seasonality_prior_scale=30)              model.add_seasonality(name='monthly', period=365.25/12, fourier_order=55)               model.add_seasonality(name='quarterly', period=365.25/4, fourier_order=5): **0.62959**
5. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.7, changepoint_range=0.8, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20): **0.57504**
6. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.7, changepoint_range=0.8, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20) & LY top down distro (np.r_[0,-392:-364] instead of np.r_[0,-28:0]): **0.64049** --> revert back to last 28 days
7. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.9, changepoint_range=0.95, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20, daily_seasonality=False): **0.58784**
8. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.7, changepoint_range=0.8, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20) & **updated data based on EDA (for CA_3, removing d1510 to 1750, for TX, limit dataset to d1100 and after, for WI, limit dataset to d650 and after)**: **0.60384**
9. Prophet(growth='linear', holidays = holidays, uncertainty_samples=False, n_changepoints = 50, changepoint_prior_scale=0.7, changepoint_range=0.8, holidays_prior_scale=20, seasonality_mode='multiplicative', seasonality_prior_scale=20) & **updated data based on EDA (for TX, limit dataset to d1100 and after, for WI, limit dataset to d650 and after)**: **0.60228**



**Modify data based on EDA**
- update data:
    - for CA_3, removing d1510 to 1750
    - for TX, limit dataset to d1100 and after
    - for WI, limit dataset to d650 and after

In [ ]:
def ModifiedCreateTimeSeries(dept_id, store_id):
    #for CA_3, removing d1510 to 1750
    #if store_id == 'CA_3':
        #df_sale_group_item_mod = df_sale_group_item.drop(df_sale_group_item.iloc[:,1512:1752], axis = 1)
        #item_series =  df_sale_group_item_mod[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
        #dates_s_mod = dates_s[0:1510] + dates_s[1750:]
        #dates = pd.DataFrame({'ds': dates_s_mod}, index=range(len(dates_s_mod)))
        #date_columns_mod = date_columns[0:1510].append(date_columns[1750:])
        #dates['y'] = item_series[date_columns_mod].values[0].transpose()     
        #return dates
    #for TX, limit dataset to d1100 and after
    if store_id in set(['TX_1', 'TX_2', 'TX_3']):
        df_sale_group_item_mod = df_sale_group_item.drop(df_sale_group_item.iloc[:,2:1101], axis = 1)
        item_series =  df_sale_group_item_mod[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
        dates_s_mod = dates_s[1099:]
        dates = pd.DataFrame({'ds': dates_s_mod}, index=range(len(dates_s_mod)))
        date_columns_mod = date_columns[1099:]
        dates['y'] = item_series[date_columns_mod].values[0].transpose()     
        return dates
    #for WI, limit dataset to d750 and after
    if store_id in set(['WI_1', 'WI_2', 'WI_3']):
        df_sale_group_item_mod = df_sale_group_item.drop(df_sale_group_item.iloc[:,2:751], axis = 1)
        item_series =  df_sale_group_item_mod[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
        dates_s_mod = dates_s[749:]
        dates = pd.DataFrame({'ds': dates_s_mod}, index=range(len(dates_s_mod)))
        date_columns_mod = date_columns[749:]
        dates['y'] = item_series[date_columns_mod].values[0].transpose()     
        return dates
    else:
        item_series =  df_sale_group_item[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
        dates = pd.DataFrame({'ds': dates_s}, index=range(len(dates_s)))
        dates['y'] = item_series[date_columns].values[0].transpose()     
        return dates

In [ ]:
def modified_run_prophet(dept_id, store_id):
    timeserie = ModifiedCreateTimeSeries(dept_id, store_id) 
    model = Prophet(growth='linear', 
                    holidays = holidays, 
                    uncertainty_samples=False, 
                    n_changepoints = 50, 
                    changepoint_prior_scale=0.7, 
                    changepoint_range=0.8, 
                    holidays_prior_scale=20, 
                    seasonality_mode='multiplicative', 
                    seasonality_prior_scale=20,
                   daily_seasonality=False)
    model.add_seasonality(name='monthly', period=365.25/12, fourier_order=55)
    model.add_seasonality(name='quarterly', period=365.25/4, fourier_order=5)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(forecast)
    return np.append(np.array([dept_id,store_id]),forecast['yhat'].values.transpose())

In [ ]:
predictions = []
for i in range(len(ids)):
    results = list(modified_run_prophet(ids[i][0],ids[i][1]))
    predictions.append(results)
    print(i)
    if i == len(ids)+1: 
        break

**Modified data** gave us a lower score -->revert back to originial dataset